In [1]:
import pandas as pd
from sqlalchemy import create_engine, text
import matplotlib.pyplot as plt
import datetime, time
import os
import json
from dotenv import load_dotenv

In [2]:

def establish_db_connection(server, database, username, password, driver):
    connection_string = (
        f"mssql+pyodbc://{username}:{password}@{server}/{database}"
        f"?driver={driver.replace(' ', '+')}"
    )
    engine = create_engine(connection_string)

    try:
        with engine.connect() as conn:
            result = conn.execute(text("SELECT @@VERSION"))
            for row in result:
                print("Connected successfully. SQL Server version:")
                print(row[0])
            return engine
    except Exception as e:
        print("Connection failed:")
        print(e)
        return None

# Adjust to include error handling for the db connection method



In [3]:
def load_env():
    load_dotenv(dotenv_path="creds\\.env")

load_env()


print(
        os.getenv('site_server'),
        os.getenv('DB_server'), 
        os.getenv('USER_NAME'),     
        os.getenv('PASSWORD_dev-test'),
        os.getenv("ODBC_DRIVER")
        )

pbs-devtest05s 158Evolution pbsuser pbs8805 ODBC Driver 17 for SQL Server


In [4]:
def SERVER_conn():
    server_conn = establish_db_connection(
        os.getenv('site_server'),
        os.getenv('DB_server'), 
        os.getenv('USER_NAME'),     
        os.getenv('PASSWORD_dev-test'),
        os.getenv("ODBC_DRIVER")
        )
    return server_conn or None 

In [5]:
server_conn = SERVER_conn()

Connected successfully. SQL Server version:
Microsoft SQL Server 2022 (RTM-CU20-GDR) (KB5065220) - 16.0.4212.1 (X64) 
	Aug 13 2025 16:40:40 
	Copyright (C) 2022 Microsoft Corporation
	Developer Edition (64-bit) on Windows Server 2022 Standard 10.0 <X64> (Build 20348: )



In [6]:
def db_request(query, server_conn_str):
    if server_conn_str is None:
        raise Exception("Database connection failed. Please check your credentials and connection settings.")

    start_time = time.time()

    respo = pd.read_sql(query, server_conn_str)

    end_time = time.time()
    print(f"Query executed in {end_time - start_time:.2f} seconds")
    return respo 

In [7]:
query_all_requests_2025_TCO6 = "SELECT *  from Ops_tblRequests where fldLastUpdated > '2025-01-1' " # removed "and fldRequestCode = 'TC06'" to get all requests"
query_all_LabourLine_2025_TCO6 = "SELECT *  from Ops_tblLabourLine where fldLastUpdated > '2025-01-1'"
query_all_PartsLine_2025_TCO6 = "SELECT *  from Ops_tblPartsLine where fldLastUpdated > '2025-01-1'"

In [ ]:
tbl_Requests_25_TC06 = db_request(query_all_requests_2025_TCO6, server_conn)
Ops_tblLabourLine_df = db_request(query_all_LabourLine_2025_TCO6, server_conn)
Ops_tblPartsLine_df = db_request(query_all_PartsLine_2025_TCO6, server_conn)

Query executed in 1.35 seconds
Query executed in 4.00 seconds
Query executed in 4.57 seconds


In [12]:
req_key_columns = ["fldId", "fldWorkItemRef", "fldSequence", "fldDescription", "fldRequestCodeRef", "fldLastUpdated"]
key_columns_for_Labour = ["fldID","fldSequence", "fldRequestRef", "fldOpCodeRef", "fldActualHours", "fldSoldHours", "fldDescription", "fldAddedDate"]
key_columns_for_Part= ["fldID", "fldSequence", "fldRequestRef", "fldSequence","fldPartNumber", "fldPartDesc", "fldPrice", "fldOrderType", "fldDateAdded"]

In [13]:
request_tbl_reduced = tbl_Requests_25_TC06[req_key_columns]
labour_tbl_reduced = Ops_tblLabourLine_df[key_columns_for_Labour]  
parts_tbl_reduced = Ops_tblPartsLine_df[key_columns_for_Part]

In [14]:
# Analyze the frequency of work item references in the request table
request_tbl_reduced["fldWorkItemRef"].value_counts().sort_values(ascending=False)


fldWorkItemRef
4BE090BD-E123-430E-8B1A-C80443F38294    2
FBAC6893-9BC2-4F19-92E2-015CDE71FC7B    1
5F2A7BB8-D7A6-457B-8990-025629701E2C    1
E5FCC3F6-E26F-43A6-A48A-027D09358BFE    1
28B6A313-A886-4D5D-82F7-042D569A10A8    1
                                       ..
5725436D-F829-455F-ACA2-E6BF84C97270    1
8B876404-AB5A-4FEE-AAC4-E6EEDC96A0BC    1
88879EC0-8286-4AFC-A216-E7337B338D8C    1
A14C81C8-8D3C-4F2A-93AB-E4F454042375    1
3B372B04-2979-4861-A749-E7429E49C44E    1
Name: count, Length: 1522, dtype: int64

In [15]:
request_tbl_reduced[request_tbl_reduced["fldWorkItemRef"] == "4BE090BD-E123-430E-8B1A-C80443F38294"]

,fldId,fldWorkItemRef,fldSequence,fldDescription,fldRequestCodeRef,fldLastUpdated
1340,BE98AE51-F102-4E10-96B3-49EFBD3F0026,4BE090BD-E123-430E-8B1A-C80443F38294,6,PERFORM BRAKE SYSTEM FLUID EXCHANGE,D3FD1B86-9CFD-455B-AD18-4B74F980F3BE,2025-01-20 14:59:25.387
1341,7F208D61-A9DE-4BDD-B886-F3031C64095F,4BE090BD-E123-430E-8B1A-C80443F38294,9,PERFORM BRAKE SYSTEM FLUID EXCHANGE,D3FD1B86-9CFD-455B-AD18-4B74F980F3BE,2025-01-20 14:59:25.390


In [16]:
parts_tbl_reduced[(parts_tbl_reduced["fldRequestRef"] == "BE98AE51-F102-4E10-96B3-49EFBD3F0026") | (parts_tbl_reduced["fldRequestRef"] == "7F208D61-A9DE-4BDD-B886-F3031C64095F")]

,fldID,fldSequence,fldRequestRef,fldSequence,fldPartNumber,fldPartDesc,fldPrice,fldOrderType,fldDateAdded
19603,A89F5100-06E3-4999-8510-99428E2D3FFA,1,BE98AE51-F102-4E10-96B3-49EFBD3F0026,1,KB44,BG 4048-84032 BRAKE SVC KIT 2PC,47.25,0,2025-01-20 21:34:19.683
19604,40AA6BBB-4730-4838-BC82-F328B04D70ED,2,BE98AE51-F102-4E10-96B3-49EFBD3F0026,2,ENV1,ENVIRO FEE,0.10,0,2025-01-20 21:37:34.503
64428,B56E4B62-D48E-4F95-8300-72118BF2553D,1,7F208D61-A9DE-4BDD-B886-F3031C64095F,1,KB44,BG 4048-84032 BRAKE SVC KIT 2PC,47.25,0,2025-01-20 21:35:23.283
64429,659ABCF7-9A4C-4598-AFEF-8BB497EB905E,2,7F208D61-A9DE-4BDD-B886-F3031C64095F,2,ENV1,ENVIRO FEE,0.10,0,2025-01-20 21:38:23.690


In [17]:
labour_tbl_reduced[(labour_tbl_reduced["fldRequestRef"] == "BE98AE51-F102-4E10-96B3-49EFBD3F0026") | (labour_tbl_reduced["fldRequestRef"] == "7F208D61-A9DE-4BDD-B886-F3031C64095F")]

,fldID,fldSequence,fldRequestRef,fldOpCodeRef,fldActualHours,fldSoldHours,fldDescription,fldAddedDate
30174,01DEC58A-B5E5-4A08-B991-122279D023E7,1,BE98AE51-F102-4E10-96B3-49EFBD3F0026,D3FD1B86-9CFD-455B-AD18-4B74F980F3BE,0.0,0.8,PERFORM BRAKE SYSTEM FLUID EXCHANGE,2025-01-20
100055,D0A3E193-1268-46CC-B511-C7A31106F912,1,7F208D61-A9DE-4BDD-B886-F3031C64095F,D3FD1B86-9CFD-455B-AD18-4B74F980F3BE,0.0,0.8,PERFORM BRAKE SYSTEM FLUID EXCHANGE,2025-01-20


#### Get a list of Labour and Parts that are associated with RequestOpCode TC06

In [18]:
# Get a list of items in parts_tbl_reduced and labour_tbl_reduced that have the sames RequestRef that is present in request_tbl_reduced
request_refs = request_tbl_reduced["fldId"].unique()
#### Get Labour and Parts that are involved by RequestOpCode
labour_involved = labour_tbl_reduced[labour_tbl_reduced["fldRequestRef"].isin(request_refs)]
parts_involved = parts_tbl_reduced[parts_tbl_reduced["fldRequestRef"].isin(request_refs)]


In [21]:
# label labour work function


def label_labour_work(search_term, df):
    df = df.copy()  # Make a safe copy to avoid SettingWithCopyWarning

    if "jobLabel" not in df.columns:
        df.loc[:, "jobLabel"] = "Other"

    df.loc[df["fldDescription"].str.contains(search_term, case=False, na=False), "jobLabel"] = search_term
    return df



In [24]:
labour_involved.head()

,fldID,fldSequence,fldRequestRef,fldOpCodeRef,fldActualHours,fldSoldHours,fldDescription,fldAddedDate
163,0E232C22-155A-40CE-9E05-ACFFB2AD6379,1,E219B6A5-B14C-4189-830B-006C1CB246EC,E1CEB931-3F90-422D-B437-CF5DB13E2E20,0.00,0.8,PERFORM BRAKE SYSTEM FLUID EXCHANGE (rec. inte...,2025-03-14
172,0C2C3888-C0CB-44D3-A66B-48DFE532F11B,1,6BF7F9D5-E0A6-4DB3-A8D2-006F528809FA,D3FD1B86-9CFD-455B-AD18-4B74F980F3BE,1.02,0.8,PERFORM BRAKE SYSTEM FLUID EXCHANGE,2025-02-03
335,ABC93739-D21C-438D-9990-3FAD74F7023F,1,44158AF8-C412-4760-AB6E-00CD2CE84D25,E1CEB931-3F90-422D-B437-CF5DB13E2E20,0.00,0.8,PERFORM BRAKE SYSTEM FLUID EXCHANGE,2025-01-08
358,40D1DC56-5DED-4B4A-AC8B-A20C5086130F,1,E4D2AE9F-ADA2-4357-B0FC-00D9156DEBD7,D3FD1B86-9CFD-455B-AD18-4B74F980F3BE,0.00,0.8,PERFORM BRAKE SYSTEM FLUID EXCHANGE,2025-01-08
394,C67E2723-D6C4-4DDA-8D4D-14710CCC211C,1,28E85955-4ECB-465F-9B6D-00ECD03344BD,7958F3C2-C11D-4EAD-A3DB-0AD7ABBA6EBF,0.00,0.8,PERFORM BRAKE SYSTEM FLUID EXCHANGE,2025-03-15


In [22]:

# label_labour_work("Brake System Fluid Exchange", labour_involved)


In [25]:
# labour_involved["jobLabel"].value_counts()

In [26]:
# get a report of how many time a unique part number appears in each unique job by fldRequestRef
parts_involved.groupby(["fldRequestRef", "fldPartNumber"]).size().reset_index(name="Count")


,fldRequestRef,fldPartNumber,Count
0,0001146D-9DF6-49FF-811E-EF88E9C7B3F0,KB44,1
1,0045CF19-B8CF-464A-BFE0-BC3F70998150,KB44,1
2,0079597F-60E2-4D38-94CC-9892419625EA,KB44,1
3,008043C1-0C45-4EEA-A38C-BA14DFB27CC4,KB44,1
4,00B1F575-4967-4FAB-9613-9CE2985CA264,KB44,1
...,...,...,...
2039,FE8F655F-0B16-4843-9CD9-583A7BAA5097,ENV1,1
2040,FE8F655F-0B16-4843-9CD9-583A7BAA5097,KB44,1
2041,FEC32FCF-AAA6-421E-BEFD-23982BA0282C,KB44,1
2042,FF10C177-0DEF-4E70-9ADD-E49DD1FACCF2,KB44,1


In [ ]:
def get_percentange_presence_parts(parts_df):
    
    if ("fldRequestRef" in parts_df.columns):
        total_jobs = parts_df["fldRequestRef"].nunique()
        
        part_presence = (
            parts_df
            .groupby("fldPartNumber")
            .agg(
                fldPartDesc=("fldPartDesc", lambda x: x.mode().iloc[0] if not x.mode().empty else None),
                JobCount=("fldRequestRef", "nunique")
            )
            .reset_index()
        )

    else:
        total_jobs = parts_df["RO.fldID"].nunique()
        
        part_presence = (
            parts_df
            .groupby("PL.fldPartNumber")
            .agg(
                fldPartDesc=("PL.fldPartDesc", lambda x: x.mode().iloc[0] if not x.mode().empty else None),
                JobCount=("RO.fldID", "nunique")
            )
            .reset_index()
        )

    part_presence["% Occurrence"] = (part_presence["JobCount"] / total_jobs) * 100
    part_presence = part_presence.sort_values(by="% Occurrence", ascending=False).reset_index(drop=True)
    return part_presence

In [30]:
def get_percentange_presence_labour(labour_df):
    

    if ("fldDescription" in labour_df.columns):
        total_jobs = labour_df["fldID"].nunique()
    # labour_presence = labour_df.groupby("fldOpCodeRef")["fldRequestRef"].nunique().reset_index(name="JobCount")
        labour_presence = (
            labour_df
            .groupby("jobLabel")
            .agg(
                fldDescription=("fldDescription", lambda x: x.mode().iloc[0] if not x.mode().empty else None),
                JobCount=("fldID", "nunique")
            )
            .reset_index()
        )
    else:
        total_jobs = labour_df["RO.fldID"].nunique()

        labour_presence = (
            labour_df
            .groupby("OpCode.fldCode")
            .agg(
                fldDescription=("LL.fldDescription", lambda x: x.mode().iloc[0] if not x.mode().empty else None),
                JobCount=("RO.fldID", "nunique")
            )
            .reset_index()
        )

    labour_presence["% Occurrence"] = (labour_presence["JobCount"] / total_jobs) * 100
    return labour_presence.sort_values(by="% Occurrence", ascending=False)


In [32]:
# get_percentange_presence_parts(parts_involved)

In [31]:
# get_percentange_presence_labour(labour_involved)

In [10]:
get_all_GOLF_Parts_Lines = "" \
"Select " \
"   RO.fldID, REQ.fldId, PL.fldID, PL.fldPartNumber, PL.fldCost, PL.fldPartRef, PL.fldShipped, PL.fldPrice, PL.fldPartDesc " \
"from " \
"Ops_tblRepairOrder RO " \
"INNER JOIN Veh_tblVehicle VEH WITH(NOLOCK) ON VEH.fldId = RO.fldVehicleRef " \
"INNER JOIN Veh_tblTrim TR WITH(NOLOCK) ON TR.fldId = VEH.fldTrimRef " \
"INNER JOIN Veh_tblModel MOD WITH(NOLOCK) ON MOD.fldId = TR.fldModelRef " \
"INNER JOIN Ops_tblRequests REQ WITH(NOLOCK) ON RO.fldId = REQ.fldWorkItemRef " \
"JOIN Ops_tblPartsLine PL WITH(nolock) ON PL.fldRequestRef = REQ.fldId " \
"WHERE 1=1 " \
"AND RO.fldStatus = 3 " \
"AND RO.fldDivision IN (1) " \
"AND MOD.fldName = 'Golf' --Filters by model name " \
"AND REQ.fldAddWorkStatus IN (100, 300, 400)  PL.fldShipped> 0"

In [11]:
get_all_GOLF_Labour_Lines = "" \
"Select  RO.fldID, REQ.fldId, LL.fldID, LL.fldOpCodeRef, LL.fldDescription, LL.fldActualHours, LL.fldAddedDate, LL.fldCost, code.fldCode, code.fldDescription, code.fldComments " \
"from Ops_tblRepairOrder RO "\
"INNER JOIN Veh_tblVehicle VEH WITH(NOLOCK) ON VEH.fldId = RO.fldVehicleRef "\
"INNER JOIN Veh_tblTrim TR WITH(NOLOCK) ON TR.fldId = VEH.fldTrimRef "\
"INNER JOIN Veh_tblModel MOD WITH(NOLOCK) ON MOD.fldId = TR.fldModelRef "\
"INNER JOIN Ops_tblRequests REQ WITH(NOLOCK) ON RO.fldId = REQ.fldWorkItemRef " \
"Join Ops_tblLabourLine LL with(NOLOCK) ON LL.fldRequestRef = REQ.fldId " \
"Join Ops_tblOpCode2 code ON code.fldID = LL.fldOpCodeRef " \
"WHERE 1=1 AND RO.fldStatus = 3 " \
    "AND RO.fldDivision IN (1) " \
    "AND MOD.fldName = 'Golf' --Filters by model name " \
    "AND REQ.fldAddWorkStatus IN (100, 300, 400) " \
    "AND REQ.fldRequestCode in ('09', '21Z06', '03Z15', '21Z06A')"

In [12]:
load_env()

In [13]:
def server_conn_174():
    server_conn_174 = establish_db_connection(
        os.getenv('site_server'),
        os.getenv('DB_server_174'), 
        os.getenv('USER_NAME'),     
        os.getenv('PASSWORD_dev-test'),
        os.getenv("ODBC_DRIVER")
        )
    return server_conn_174 or None

In [14]:
load_env()


print(
        os.getenv('site_server'),
        os.getenv('DB_server_174'), 
        os.getenv('USER_NAME'),     
        os.getenv('PASSWORD_dev-test'),
        os.getenv("ODBC_DRIVER")
        )

pbs-devtest05s 174Evolution pbsuser pbs8805 ODBC Driver 17 for SQL Server


In [15]:
server_conn_174 = server_conn_174()

Connected successfully. SQL Server version:
Microsoft SQL Server 2022 (RTM-CU20-GDR) (KB5065220) - 16.0.4212.1 (X64) 
	Aug 13 2025 16:40:40 
	Copyright (C) 2022 Microsoft Corporation
	Developer Edition (64-bit) on Windows Server 2022 Standard 10.0 <X64> (Build 20348: )



In [16]:
GOLF_Parts_Lines_df = db_request(get_all_GOLF_Parts_Lines, server_conn_174)


Query executed in 7.38 seconds


In [17]:
GOLF_Labour_Lines_df = db_request(get_all_GOLF_Labour_Lines, server_conn_174)

Query executed in 6.42 seconds


In [18]:
len(GOLF_Labour_Lines_df.columns)

11

In [19]:
GOLF_Labour_Lines_df.columns = ["RO.fldID","REQ.fldId","LL.fldID","LL.fldOpCodeRef","LL.fldDescription","LL.fldActualHours","LL.fldAddedDate","LL.fldCost", "OpCode.fldCode", "OpCode.fldDescription", "OpCode.fldComments"]


In [22]:
GOLF_Parts_Lines_df.columns = ["RO.fldID","REQ.fldId", "PL.fldID,", "PL.fldPartNumber","PL.fldCost","PL.fldPartRef","PL.fldShipped","PL.fldPrice","PL.fldPartDesc"]

In [23]:
print(f'Total unique ROs Labour df: {GOLF_Labour_Lines_df["RO.fldID"].count()}')
print(f'Total unique ROs Parts df: {GOLF_Parts_Lines_df["RO.fldID"].count()}')

Total unique ROs Labour df: 86463
Total unique ROs Parts df: 98804


In [24]:

print(f'Total unique Reqs Labour_df: {GOLF_Labour_Lines_df["REQ.fldId"].count()}')
print(f'Total unique Reqs Parts_df: {GOLF_Parts_Lines_df["REQ.fldId"].count()}')
 

Total unique Reqs Labour_df: 86463
Total unique Reqs Parts_df: 98804


In [25]:
GOLF_Labour_Lines_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86463 entries, 0 to 86462
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   RO.fldID               86463 non-null  object        
 1   REQ.fldId              86463 non-null  object        
 2   LL.fldID               86463 non-null  object        
 3   LL.fldOpCodeRef        86463 non-null  object        
 4   LL.fldDescription      86463 non-null  object        
 5   LL.fldActualHours      86463 non-null  float64       
 6   LL.fldAddedDate        25693 non-null  datetime64[ns]
 7   LL.fldCost             86463 non-null  float64       
 8   OpCode.fldCode         86463 non-null  object        
 9   OpCode.fldDescription  86463 non-null  object        
 10  OpCode.fldComments     82259 non-null  object        
dtypes: datetime64[ns](1), float64(2), object(8)
memory usage: 7.3+ MB


In [26]:

GOLF_Parts_Lines_df["PL.fldPartNumber"].value_counts().sort_values(ascending=False)


PL.fldPartNumber
ENV55              6594
ENV5               6297
06L 103 801        3890
N 908 132 02       3668
06L 115 562 B      3400
                   ... 
1K0 881 449           1
5GM 882 252 82V       1
5QM 407 152           1
1K0 423 831 A         1
5QM 513 049           1
Name: count, Length: 4472, dtype: int64

In [33]:
parts_perc_res = get_percentange_presence_parts(GOLF_Parts_Lines_df)

In [34]:
parts_perc_res.head(25)

,PL.fldPartNumber,fldPartDesc,JobCount,% Occurrence
0,ENV55,ENVIRO FEE,6563,42.733429
1,ENV5,ENVIRO FEE,6271,40.832140
2,06L 103 801,drain plug,3884,25.289751
3,N 908 132 02,SCREW,3649,23.759604
4,06L 115 562 B,FILTERELEM,3391,22.079698
5,06D 115 562,FILTERELEM,1993,12.976950
6,ZVW 352 540 S DSP,ENGINE OIL,1887,12.286756
7,G E55 540 1B DSP,ENGINE OIL,1868,12.163042
8,G E52 502 1B DSP,EDGE,1794,11.681208
9,84032,BG BRAKE FLUID,1178,7.670270


In [234]:
labour_perc_res =get_percentange_presence_labour(GOLF_Labour_Lines_df)

In [235]:
labour_perc_res.head(25)

,OpCode.fldCode,fldDescription,JobCount,% Occurrence
145,09,GENERAL MECHANICAL,2614,12.629240
134,07Z2,SWAP WINTER FOR SUMM ...,2177,10.517924
630,25,Auto complete inspection,1820,8.793120
56,03Z15,BG BRAKE FLUID SERVI ...,1731,8.363127
543,21Z06,AIR FILTER ...,1672,8.078075
544,21Z06A,POLLEN FILTER ...,1578,7.623925
45,03Z01,OIL CHANGE ...,1295,6.256643
128,07Z1,SWAP SUMMER TO WINTER ...,1269,6.131027
16,01Z01,OIL CHANGE/SYNTHETIC ...,964,4.657455
58,03Z17,BG COOLANT SERVICE ...,957,4.623635


In [236]:
GOLF_Labour_Lines_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86463 entries, 0 to 86462
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   RO.fldID               86463 non-null  object        
 1   REQ.fldId              86463 non-null  object        
 2   LL.fldID               86463 non-null  object        
 3   LL.fldOpCodeRef        86463 non-null  object        
 4   LL.fldDescription      86463 non-null  object        
 5   LL.fldActualHours      86463 non-null  float64       
 6   LL.fldAddedDate        25693 non-null  datetime64[ns]
 7   LL.fldCost             86463 non-null  float64       
 8   OpCode.fldCode         86463 non-null  object        
 9   OpCode.fldDescription  86463 non-null  object        
 10  OpCode.fldComments     82259 non-null  object        
dtypes: datetime64[ns](1), float64(2), object(8)
memory usage: 7.3+ MB


In [237]:
GOLF_Labour_Lines_df["RO.fldID"].value_counts()

RO.fldID
0C0E943A-25C6-40E0-8B3E-30D8A5E14030    98
3FE4E8B4-2889-4591-9AD2-A835022FD3A5    63
2B63BBEC-93AC-4649-A2CE-79F4009102B5    57
B8B28DE4-92F3-4929-8242-A250D55A69CE    49
616D4EFA-E85B-4791-984F-74605F498E85    47
                                        ..
94CB40B7-E755-4826-87FB-B62547A3B941     1
93A93B90-CF09-49B2-AA6D-FAF6230975BA     1
5E2F4101-9F21-4BA6-9E12-A88D1036A353     1
21E67539-C5C4-404C-B65C-9A61743BE79E     1
3E9F4366-F928-4E2C-9B25-6DD5243253C5     1
Name: count, Length: 20698, dtype: int64

In [238]:
GOLF_Labour_Lines_df[GOLF_Labour_Lines_df["RO.fldID"]=="0C0E943A-25C6-40E0-8B3E-30D8A5E14030"].tail()

,RO.fldID,REQ.fldId,LL.fldID,LL.fldOpCodeRef,LL.fldDescription,LL.fldActualHours,LL.fldAddedDate,LL.fldCost,OpCode.fldCode,OpCode.fldDescription,OpCode.fldComments
67497,0C0E943A-25C6-40E0-8B3E-30D8A5E14030,BA5BCC4E-8B80-45E8-BFC6-149DF2D961BE,9B16566E-6C10-49E2-97DC-01038796E5A2,3BDD486D-9308-4632-A1AF-BEF74C6F5AEB,"64,000 KM MAINT. ...",0.0,NaT,0.0,10B064V1,"64,000 KM MAINT. ...",PERFORM OIL & FILTER CHANGE. CHECK & ADJUST TI...
67498,0C0E943A-25C6-40E0-8B3E-30D8A5E14030,BA5BCC4E-8B80-45E8-BFC6-149DF2D961BE,7C83C9B7-6983-4A61-9B89-B093E4CF5C4E,3BDD486D-9308-4632-A1AF-BEF74C6F5AEB,"64,000 KM MAINT. ...",0.0,NaT,0.0,10B064V1,"64,000 KM MAINT. ...",PERFORM OIL & FILTER CHANGE. CHECK & ADJUST TI...
67499,0C0E943A-25C6-40E0-8B3E-30D8A5E14030,BA5BCC4E-8B80-45E8-BFC6-149DF2D961BE,4A407043-1982-4EEA-B06A-82F96DD7862E,3BDD486D-9308-4632-A1AF-BEF74C6F5AEB,"64,000 KM MAINT. ...",0.0,NaT,0.0,10B064V1,"64,000 KM MAINT. ...",PERFORM OIL & FILTER CHANGE. CHECK & ADJUST TI...
67500,0C0E943A-25C6-40E0-8B3E-30D8A5E14030,BA5BCC4E-8B80-45E8-BFC6-149DF2D961BE,882E2E98-4DB0-42DC-B0BA-F5006E07D399,3BDD486D-9308-4632-A1AF-BEF74C6F5AEB,"64,000 KM MAINT. ...",1.5,NaT,0.0,10B064V1,"64,000 KM MAINT. ...",PERFORM OIL & FILTER CHANGE. CHECK & ADJUST TI...
67501,0C0E943A-25C6-40E0-8B3E-30D8A5E14030,BA5BCC4E-8B80-45E8-BFC6-149DF2D961BE,EEB90298-223D-4885-BE71-A00B98F0637B,3BDD486D-9308-4632-A1AF-BEF74C6F5AEB,"64,000 KM MAINT. ...",0.0,NaT,0.0,10B064V1,"64,000 KM MAINT. ...",PERFORM OIL & FILTER CHANGE. CHECK & ADJUST TI...


In [239]:
GOLF_Labour_Lines_df["REQ.fldId"].value_counts()

REQ.fldId
83174AAF-1EE6-4CC7-9434-15C8A10C9AC5    58
80C3246E-4DFC-41E5-AAEF-6DF68D40ECDE    44
F6E5DF57-E20A-487F-9419-A0E3563A8548    31
91E2822C-BE65-40C5-9C47-AAEFD4F2CE1A    30
C251D91B-868C-4B62-A42E-60BF68E897D3    30
                                        ..
36005D8A-FE4A-42B6-BF44-2CF2D6D55E96     1
6C4DAD1B-CE86-4BF6-AEF6-376F9A8DECF8     1
060D154A-2499-4159-BF94-50AB976DCD67     1
09EE3C50-FA97-4681-A325-4254DADF5440     1
52EC80BF-EF72-4696-A034-6649F2069454     1
Name: count, Length: 60125, dtype: int64

In [240]:
GOLF_Labour_Lines_df[GOLF_Labour_Lines_df["REQ.fldId"]=="52EC80BF-EF72-4696-A034-6649F2069454"]

,RO.fldID,REQ.fldId,LL.fldID,LL.fldOpCodeRef,LL.fldDescription,LL.fldActualHours,LL.fldAddedDate,LL.fldCost,OpCode.fldCode,OpCode.fldDescription,OpCode.fldComments
86462,CEE863EC-83AD-4975-A884-C0E051CDD08F,52EC80BF-EF72-4696-A034-6649F2069454,D51FCCAA-2A69-44F5-AE2D-DE32905863DE,9EB717BD-AA08-4722-A1DC-FFC84E948869,SEASONAL TIRE SPECIAL,0.46,2022-05-02,9.66,07SEASON,SEASONAL TIRE SPECIAL,GET READY FOR SPRING|\nINCLUDES: TIRE SWAP (ON...


In [241]:
select_dataset_top_3_opcode = GOLF_Labour_Lines_df[GOLF_Labour_Lines_df["OpCode.fldCode"].isin(["03Z15","21Z06A","03Z01"])]

In [242]:
select_dataset_top_3_opcode["REQ.fldId"].value_counts()

REQ.fldId
0C78FC90-D985-4B6E-8D33-74B045D52F75    4
DE0AE3F8-CC6A-44C0-BBAB-8EC8673E0236    4
2D4898F3-C71F-4BC4-B501-5A1E39815B6B    4
CE013DB8-6B41-467B-9382-A6323295ED7A    3
C60CE6AD-7E73-4979-A565-7716BCB19191    3
                                       ..
9059C975-686B-4DCA-A9ED-BB1D424E7AC6    1
336C0E06-E947-41B6-AFE6-BC379F58BF94    1
F625102B-45EA-44D9-B3B0-BC70D744BF8E    1
19864E60-A3D4-4EC2-83C9-B5ED33B6BC2D    1
E046AC22-ACB8-4C64-86EE-9FB1B3F39823    1
Name: count, Length: 4606, dtype: int64

In [243]:
# select_dataset_top_3_opcode[select_dataset_top_3_opcode["REQ.fldId"]=="4B80A58E-98EE-4F98-98C0-11FB0F148F63"][]

In [244]:
select_dataset_top_3_opcode.head(5)

,RO.fldID,REQ.fldId,LL.fldID,LL.fldOpCodeRef,LL.fldDescription,LL.fldActualHours,LL.fldAddedDate,LL.fldCost,OpCode.fldCode,OpCode.fldDescription,OpCode.fldComments
1480,54120850-E6E1-4DB7-B3C7-9DD02E8F81E3,0836AB11-7720-4356-BAE9-72FD08B69CD9,39F26BC4-ED16-4DB1-8B74-EAEC6438B98D,C89585FD-576F-49F7-BC6A-048A3A45421B,BG BRAKE FLUID SERVI ...,0.66,2022-05-25,16.5,03Z15,BG BRAKE FLUID SERVICE $159.99 + GST & SS,"PERFORM BG BRAKE FLUID FLUSH. FLUSH ALL LINES,..."
1481,BDB5BC8E-30C8-4332-BDAC-C95F293562DC,22C22F12-B21C-4EF9-B926-710DF9FD54B3,4539A485-4B9B-406B-9B2D-77CA3670A991,C89585FD-576F-49F7-BC6A-048A3A45421B,BG BRAKE FLUID SERVICE,0.42,2023-08-18,31.5,03Z15,BG BRAKE FLUID SERVICE $159.99 + GST & SS,"PERFORM BG BRAKE FLUID FLUSH. FLUSH ALL LINES,..."
1482,C63B6564-A70F-4B18-A4D0-BAF972CA3E1F,19964989-E10C-414F-81B8-710EFD0AFD6F,F44EC2D1-5F72-4EC4-A248-276CDCF57567,C89585FD-576F-49F7-BC6A-048A3A45421B,BG BRAKE FLUID SERVICE $159.99 + GST & SS,0.77,2025-02-27,36.9,03Z15,BG BRAKE FLUID SERVICE $159.99 + GST & SS,"PERFORM BG BRAKE FLUID FLUSH. FLUSH ALL LINES,..."
1483,79C475D3-6897-46A6-8A07-CC8E0E3B1E8A,4A57302F-4368-43CC-A79E-7121597503F4,AEBB087B-139C-48E5-B61A-EE5E13409ACE,C89585FD-576F-49F7-BC6A-048A3A45421B,BG BRAKE FLUID SERVICE $159.99 + GST & SS \r\n...,0.00,2024-05-21,0.0,03Z15,BG BRAKE FLUID SERVICE $159.99 + GST & SS,"PERFORM BG BRAKE FLUID FLUSH. FLUSH ALL LINES,..."
1484,4B0D831E-3B88-4CB2-964F-D2C3EBDB9F47,E88D4294-94FF-4EFC-A5D4-700686850758,7ABDEB8C-CC89-479F-ACF5-96E9280034E0,C89585FD-576F-49F7-BC6A-048A3A45421B,BG BRAKE FLUID SERVICE,0.37,2022-04-07,18.9,03Z15,BG BRAKE FLUID SERVICE $159.99 + GST & SS,"PERFORM BG BRAKE FLUID FLUSH. FLUSH ALL LINES,..."


In [245]:
select_dataset_top_3_opcode[select_dataset_top_3_opcode["LL.fldActualHours"]>0].head()

,RO.fldID,REQ.fldId,LL.fldID,LL.fldOpCodeRef,LL.fldDescription,LL.fldActualHours,LL.fldAddedDate,LL.fldCost,OpCode.fldCode,OpCode.fldDescription,OpCode.fldComments
1480,54120850-E6E1-4DB7-B3C7-9DD02E8F81E3,0836AB11-7720-4356-BAE9-72FD08B69CD9,39F26BC4-ED16-4DB1-8B74-EAEC6438B98D,C89585FD-576F-49F7-BC6A-048A3A45421B,BG BRAKE FLUID SERVI ...,0.66,2022-05-25,16.5,03Z15,BG BRAKE FLUID SERVICE $159.99 + GST & SS,"PERFORM BG BRAKE FLUID FLUSH. FLUSH ALL LINES,..."
1481,BDB5BC8E-30C8-4332-BDAC-C95F293562DC,22C22F12-B21C-4EF9-B926-710DF9FD54B3,4539A485-4B9B-406B-9B2D-77CA3670A991,C89585FD-576F-49F7-BC6A-048A3A45421B,BG BRAKE FLUID SERVICE,0.42,2023-08-18,31.5,03Z15,BG BRAKE FLUID SERVICE $159.99 + GST & SS,"PERFORM BG BRAKE FLUID FLUSH. FLUSH ALL LINES,..."
1482,C63B6564-A70F-4B18-A4D0-BAF972CA3E1F,19964989-E10C-414F-81B8-710EFD0AFD6F,F44EC2D1-5F72-4EC4-A248-276CDCF57567,C89585FD-576F-49F7-BC6A-048A3A45421B,BG BRAKE FLUID SERVICE $159.99 + GST & SS,0.77,2025-02-27,36.9,03Z15,BG BRAKE FLUID SERVICE $159.99 + GST & SS,"PERFORM BG BRAKE FLUID FLUSH. FLUSH ALL LINES,..."
1484,4B0D831E-3B88-4CB2-964F-D2C3EBDB9F47,E88D4294-94FF-4EFC-A5D4-700686850758,7ABDEB8C-CC89-479F-ACF5-96E9280034E0,C89585FD-576F-49F7-BC6A-048A3A45421B,BG BRAKE FLUID SERVICE,0.37,2022-04-07,18.9,03Z15,BG BRAKE FLUID SERVICE $159.99 + GST & SS,"PERFORM BG BRAKE FLUID FLUSH. FLUSH ALL LINES,..."
1486,B4F13897-7532-49B3-9CF1-9D75F0261CE2,28CBE0FE-F6A0-4797-AE10-704C7C7C1B35,0A4BA1D4-0BBC-498F-BBDE-D85576DA3C0D,C89585FD-576F-49F7-BC6A-048A3A45421B,BG BRAKE FLUID SERVI ...,0.42,NaT,0.0,03Z15,BG BRAKE FLUID SERVICE $159.99 + GST & SS,"PERFORM BG BRAKE FLUID FLUSH. FLUSH ALL LINES,..."


In [246]:
select_dataset_top_3_opcode[select_dataset_top_3_opcode["REQ.fldId"]=="2D4898F3-C71F-4BC4-B501-5A1E39815B6B"]

,RO.fldID,REQ.fldId,LL.fldID,LL.fldOpCodeRef,LL.fldDescription,LL.fldActualHours,LL.fldAddedDate,LL.fldCost,OpCode.fldCode,OpCode.fldDescription,OpCode.fldComments
2539,77BEF492-E384-4B16-AD4A-2AF6ABFB8E49,2D4898F3-C71F-4BC4-B501-5A1E39815B6B,8B9CBC17-DE2A-4251-AF3C-2FAAE8EFB1D4,C89585FD-576F-49F7-BC6A-048A3A45421B,BG BRAKE FLUID SERVI ...,0.0,NaT,0.0,03Z15,BG BRAKE FLUID SERVICE $159.99 + GST & SS,"PERFORM BG BRAKE FLUID FLUSH. FLUSH ALL LINES,..."
2540,77BEF492-E384-4B16-AD4A-2AF6ABFB8E49,2D4898F3-C71F-4BC4-B501-5A1E39815B6B,73DD821E-A22D-4BDB-948F-2022600427DC,C89585FD-576F-49F7-BC6A-048A3A45421B,BG BRAKE FLUID SERVI ...,0.0,NaT,0.0,03Z15,BG BRAKE FLUID SERVICE $159.99 + GST & SS,"PERFORM BG BRAKE FLUID FLUSH. FLUSH ALL LINES,..."
2541,77BEF492-E384-4B16-AD4A-2AF6ABFB8E49,2D4898F3-C71F-4BC4-B501-5A1E39815B6B,1EF28E97-50B4-483A-9772-D431932AC63C,C89585FD-576F-49F7-BC6A-048A3A45421B,BG BRAKE FLUID SERVI ...,0.0,NaT,0.0,03Z15,BG BRAKE FLUID SERVICE $159.99 + GST & SS,"PERFORM BG BRAKE FLUID FLUSH. FLUSH ALL LINES,..."
2542,77BEF492-E384-4B16-AD4A-2AF6ABFB8E49,2D4898F3-C71F-4BC4-B501-5A1E39815B6B,D0637D7F-5A7C-4EE1-B495-0FAA96110D9E,C89585FD-576F-49F7-BC6A-048A3A45421B,BG BRAKE FLUID SERVI ...,0.9,NaT,0.0,03Z15,BG BRAKE FLUID SERVICE $159.99 + GST & SS,"PERFORM BG BRAKE FLUID FLUSH. FLUSH ALL LINES,..."


In [247]:
GOLF_Parts_Lines_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98804 entries, 0 to 98803
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   RO.fldID          98804 non-null  object 
 1   REQ.fldId         98804 non-null  object 
 2   PL.fldID,         98804 non-null  object 
 3   PL.fldPartNumber  98804 non-null  object 
 4   PL.fldCost        98804 non-null  float64
 5   PL.fldPartRef     98804 non-null  object 
 6   PL.fldShipped     98804 non-null  float64
 7   PL.fldPrice       98804 non-null  float64
 8   PL.fldPartDesc    98804 non-null  object 
dtypes: float64(3), object(6)
memory usage: 6.8+ MB


In [248]:

select_dataset_top_3_opcode[select_dataset_top_3_opcode["REQ.fldId"]=="2D4898F3-C71F-4BC4-B501-5A1E39815B6B"] 
 

,RO.fldID,REQ.fldId,LL.fldID,LL.fldOpCodeRef,LL.fldDescription,LL.fldActualHours,LL.fldAddedDate,LL.fldCost,OpCode.fldCode,OpCode.fldDescription,OpCode.fldComments
2539,77BEF492-E384-4B16-AD4A-2AF6ABFB8E49,2D4898F3-C71F-4BC4-B501-5A1E39815B6B,8B9CBC17-DE2A-4251-AF3C-2FAAE8EFB1D4,C89585FD-576F-49F7-BC6A-048A3A45421B,BG BRAKE FLUID SERVI ...,0.0,NaT,0.0,03Z15,BG BRAKE FLUID SERVICE $159.99 + GST & SS,"PERFORM BG BRAKE FLUID FLUSH. FLUSH ALL LINES,..."
2540,77BEF492-E384-4B16-AD4A-2AF6ABFB8E49,2D4898F3-C71F-4BC4-B501-5A1E39815B6B,73DD821E-A22D-4BDB-948F-2022600427DC,C89585FD-576F-49F7-BC6A-048A3A45421B,BG BRAKE FLUID SERVI ...,0.0,NaT,0.0,03Z15,BG BRAKE FLUID SERVICE $159.99 + GST & SS,"PERFORM BG BRAKE FLUID FLUSH. FLUSH ALL LINES,..."
2541,77BEF492-E384-4B16-AD4A-2AF6ABFB8E49,2D4898F3-C71F-4BC4-B501-5A1E39815B6B,1EF28E97-50B4-483A-9772-D431932AC63C,C89585FD-576F-49F7-BC6A-048A3A45421B,BG BRAKE FLUID SERVI ...,0.0,NaT,0.0,03Z15,BG BRAKE FLUID SERVICE $159.99 + GST & SS,"PERFORM BG BRAKE FLUID FLUSH. FLUSH ALL LINES,..."
2542,77BEF492-E384-4B16-AD4A-2AF6ABFB8E49,2D4898F3-C71F-4BC4-B501-5A1E39815B6B,D0637D7F-5A7C-4EE1-B495-0FAA96110D9E,C89585FD-576F-49F7-BC6A-048A3A45421B,BG BRAKE FLUID SERVI ...,0.9,NaT,0.0,03Z15,BG BRAKE FLUID SERVICE $159.99 + GST & SS,"PERFORM BG BRAKE FLUID FLUSH. FLUSH ALL LINES,..."


In [249]:
print(GOLF_Parts_Lines_df[GOLF_Parts_Lines_df["REQ.fldId"]=="2D4898F3-C71F-4BC4-B501-5A1E39815B6B"][["PL.fldPartNumber", "PL.fldPartDesc", "PL.fldPrice"]])

      PL.fldPartNumber  PL.fldPartDesc  PL.fldPrice
19194            84032  BG BRAKE FLUID        27.66


In [250]:
def get_labourAndParts_by_ReqFlId(ReqFlId):
    print("Labour Items -------------------------")
    labour_res = select_dataset_top_3_opcode[select_dataset_top_3_opcode["REQ.fldId"]==ReqFlId][["OpCode.fldCode", "LL.fldDescription"]]
    labour_perc_res.reset_index(drop=True, inplace=True)
    
    print(labour_res) 

    print("Part Items -------------------------")
    parts_res = GOLF_Parts_Lines_df[GOLF_Parts_Lines_df["REQ.fldId"]==ReqFlId][["PL.fldPartNumber", "PL.fldPartDesc", "PL.fldPrice"]]
    parts_res.reset_index(drop=True, inplace =True)
    print(parts_res)


In [251]:
get_labourAndParts_by_ReqFlId("2D4898F3-C71F-4BC4-B501-5A1E39815B6B")

Labour Items -------------------------
     OpCode.fldCode                                  LL.fldDescription
2539          03Z15  BG BRAKE FLUID SERVI                          ...
2540          03Z15  BG BRAKE FLUID SERVI                          ...
2541          03Z15  BG BRAKE FLUID SERVI                          ...
2542          03Z15  BG BRAKE FLUID SERVI                          ...
Part Items -------------------------
  PL.fldPartNumber  PL.fldPartDesc  PL.fldPrice
0            84032  BG BRAKE FLUID        27.66


In [252]:
def get_labourAndParts_by_ROFldId(ROFlId):
    print("Labour Items -------------------------")
    labour_res = select_dataset_top_3_opcode[select_dataset_top_3_opcode["RO.fldID"]==ROFlId][["OpCode.fldCode", "LL.fldDescription"]]
    labour_perc_res.reset_index(drop=True, inplace=True)
    
    print(labour_res) 

    print("Part Items -------------------------")
    parts_res = GOLF_Parts_Lines_df[GOLF_Parts_Lines_df["RO.fldID"]==ROFlId][["PL.fldPartNumber", "PL.fldPartDesc", "PL.fldPrice"]]
    parts_res.reset_index(drop=True, inplace =True)
    print(parts_res)


In [253]:
get_labourAndParts_by_ROFldId("77BEF492-E384-4B16-AD4A-2AF6ABFB8E49")

Labour Items -------------------------
     OpCode.fldCode                                  LL.fldDescription
2539          03Z15  BG BRAKE FLUID SERVI                          ...
2540          03Z15  BG BRAKE FLUID SERVI                          ...
2541          03Z15  BG BRAKE FLUID SERVI                          ...
2542          03Z15  BG BRAKE FLUID SERVI                          ...
Part Items -------------------------
  PL.fldPartNumber  PL.fldPartDesc  PL.fldPrice
0    1K0 129 620 L       AIRFILTER        22.68
1              54A    64KM SERVICE         0.00
2     N 908 132 02           SCREW         3.52
3    JZW 819 653 B      FILTER INS        19.77
4    071 115 562 C      FILTERELEM        16.79
5    1K0 127 434 B      FILTERELEM        46.61
6        G0521951Q         ENG OIL        10.07
7            84032  BG BRAKE FLUID        27.66


In [254]:
def all_parts__by_OpCode(OpCode):

    # query for all labour items for the opcode
    list_of_labout_items_by_opCode = select_dataset_top_3_opcode[select_dataset_top_3_opcode["OpCode.fldCode"]==OpCode][["LL.fldDescription", "RO.fldID", "REQ.fldId", "LL.fldID"]]

    # grab all the RO_Ids to search for parts
    RO_ID_to_search_in_for_parts = list_of_labout_items_by_opCode["RO.fldID"]

    # Query parts items with the same RO_ID
    parts_items_per_opCode = GOLF_Parts_Lines_df[GOLF_Parts_Lines_df["RO.fldID"].isin(RO_ID_to_search_in_for_parts)]

    # 
    parts_items_perc_by_opcode = get_percentange_presence_parts(parts_items_per_opCode)

    # removed fees from the list

    parts_items_perc_drop_noise = parts_items_perc_by_opcode[~parts_items_perc_by_opcode["fldPartDesc"].str.contains("FEE", case=False, na=False)]
    return parts_items_perc_drop_noise, list_of_labout_items_by_opCode



In [255]:
list_of_labout_items_by_opCode = select_dataset_top_3_opcode[select_dataset_top_3_opcode["OpCode.fldCode"]=="03Z15"][["LL.fldDescription", "OpCode.fldCode", "RO.fldID", "REQ.fldId", "LL.fldID"]]

In [256]:
list_of_labout_items_by_opCode.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1989 entries, 1480 to 3468
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   LL.fldDescription  1989 non-null   object
 1   OpCode.fldCode     1989 non-null   object
 2   RO.fldID           1989 non-null   object
 3   REQ.fldId          1989 non-null   object
 4   LL.fldID           1989 non-null   object
dtypes: object(5)
memory usage: 93.2+ KB


In [257]:
get_percentange_presence_labour(list_of_labout_items_by_opCode)

,OpCode.fldCode,fldDescription,JobCount,% Occurrence
0,03Z15,BG BRAKE FLUID SERVI ...,1731,100.0


In [258]:
GOLF_Labour_Lines_df[GOLF_Labour_Lines_df["OpCode.fldCode"]=="03Z15"].head(20)

,RO.fldID,REQ.fldId,LL.fldID,LL.fldOpCodeRef,LL.fldDescription,LL.fldActualHours,LL.fldAddedDate,LL.fldCost,OpCode.fldCode,OpCode.fldDescription,OpCode.fldComments
1480,54120850-E6E1-4DB7-B3C7-9DD02E8F81E3,0836AB11-7720-4356-BAE9-72FD08B69CD9,39F26BC4-ED16-4DB1-8B74-EAEC6438B98D,C89585FD-576F-49F7-BC6A-048A3A45421B,BG BRAKE FLUID SERVI ...,0.66,2022-05-25,16.5,03Z15,BG BRAKE FLUID SERVICE $159.99 + GST & SS,"PERFORM BG BRAKE FLUID FLUSH. FLUSH ALL LINES,..."
1481,BDB5BC8E-30C8-4332-BDAC-C95F293562DC,22C22F12-B21C-4EF9-B926-710DF9FD54B3,4539A485-4B9B-406B-9B2D-77CA3670A991,C89585FD-576F-49F7-BC6A-048A3A45421B,BG BRAKE FLUID SERVICE,0.42,2023-08-18,31.5,03Z15,BG BRAKE FLUID SERVICE $159.99 + GST & SS,"PERFORM BG BRAKE FLUID FLUSH. FLUSH ALL LINES,..."
1482,C63B6564-A70F-4B18-A4D0-BAF972CA3E1F,19964989-E10C-414F-81B8-710EFD0AFD6F,F44EC2D1-5F72-4EC4-A248-276CDCF57567,C89585FD-576F-49F7-BC6A-048A3A45421B,BG BRAKE FLUID SERVICE $159.99 + GST & SS,0.77,2025-02-27,36.9,03Z15,BG BRAKE FLUID SERVICE $159.99 + GST & SS,"PERFORM BG BRAKE FLUID FLUSH. FLUSH ALL LINES,..."
1483,79C475D3-6897-46A6-8A07-CC8E0E3B1E8A,4A57302F-4368-43CC-A79E-7121597503F4,AEBB087B-139C-48E5-B61A-EE5E13409ACE,C89585FD-576F-49F7-BC6A-048A3A45421B,BG BRAKE FLUID SERVICE $159.99 + GST & SS \r\n...,0.00,2024-05-21,0.0,03Z15,BG BRAKE FLUID SERVICE $159.99 + GST & SS,"PERFORM BG BRAKE FLUID FLUSH. FLUSH ALL LINES,..."
1484,4B0D831E-3B88-4CB2-964F-D2C3EBDB9F47,E88D4294-94FF-4EFC-A5D4-700686850758,7ABDEB8C-CC89-479F-ACF5-96E9280034E0,C89585FD-576F-49F7-BC6A-048A3A45421B,BG BRAKE FLUID SERVICE,0.37,2022-04-07,18.9,03Z15,BG BRAKE FLUID SERVICE $159.99 + GST & SS,"PERFORM BG BRAKE FLUID FLUSH. FLUSH ALL LINES,..."
1485,7988E0E4-3CC5-4588-A13C-AC44D2352502,38C9E9E2-E35C-4EA6-84F1-5539647F7C78,FF0F9C66-3CFA-46E4-8FC7-7EF315E1DA65,C89585FD-576F-49F7-BC6A-048A3A45421B,BG BRAKE FLUID SERVICE $159.99 + GST & SS,0.00,2024-03-01,0.0,03Z15,BG BRAKE FLUID SERVICE $159.99 + GST & SS,"PERFORM BG BRAKE FLUID FLUSH. FLUSH ALL LINES,..."
1486,B4F13897-7532-49B3-9CF1-9D75F0261CE2,28CBE0FE-F6A0-4797-AE10-704C7C7C1B35,0A4BA1D4-0BBC-498F-BBDE-D85576DA3C0D,C89585FD-576F-49F7-BC6A-048A3A45421B,BG BRAKE FLUID SERVI ...,0.42,NaT,0.0,03Z15,BG BRAKE FLUID SERVICE $159.99 + GST & SS,"PERFORM BG BRAKE FLUID FLUSH. FLUSH ALL LINES,..."
1487,9B373A7A-61AB-40DF-B21C-B356207EB28F,1ACF826E-3A2A-4B09-981A-76CCA9156DA6,106E82CC-6A0F-4C5F-BAF6-EDA3181D4E24,C89585FD-576F-49F7-BC6A-048A3A45421B,BG BRAKE FLUID SERVICE,0.00,2023-10-20,0.0,03Z15,BG BRAKE FLUID SERVICE $159.99 + GST & SS,"PERFORM BG BRAKE FLUID FLUSH. FLUSH ALL LINES,..."
1488,779AD8CB-B842-43C0-913C-9AE7732C531F,501B6B33-40F6-463D-8E95-6882B7982C5C,9B433088-64EF-4459-A045-EE3849C02B0B,C89585FD-576F-49F7-BC6A-048A3A45421B,BG BRAKE FLUID SERVI ...,0.89,2022-02-04,22.5,03Z15,BG BRAKE FLUID SERVICE $159.99 + GST & SS,"PERFORM BG BRAKE FLUID FLUSH. FLUSH ALL LINES,..."
1489,8A02BE22-B1CA-4C44-9658-C044DC76E882,1ECB726E-4870-4499-BA82-683F32AAE815,948852B8-1986-4349-ABD8-2A3D4A83AA67,C89585FD-576F-49F7-BC6A-048A3A45421B,BG BRAKE FLUID SERVI ...,0.29,2022-01-29,18.9,03Z15,BG BRAKE FLUID SERVICE $159.99 + GST & SS,"PERFORM BG BRAKE FLUID FLUSH. FLUSH ALL LINES,..."


In [259]:
parts, labor = all_parts__by_OpCode("03Z15")

In [260]:
parts.head(10)

,PL.fldPartNumber,fldPartDesc,JobCount,% Occurrence
2,84032,BG BRAKE FLUID,1058,61.156069
3,06L 115 562 B,FILTERELEM,788,45.549133
4,06L 103 801,drain plug,619,35.780347
5,N 908 132 02,SCREW,613,35.433526
6,G E55 540 1B DSP,ENGINE OIL,520,30.057803
7,B 000 750 1L DSP,BRAKEFLUID,469,27.109827
8,5Q0 819 669,Od. filter,465,26.878613
9,5QM 129 620,AIRFILTER,424,24.508671
10,K5902,FLUSH KIT,360,20.809249
11,06D 115 562,FILTERELEM,323,18.670520


- Find parts list and labour by part-number
- Start with parts and move up to the service job being done
- 


- Search of part, if it's there
- Look at part number occurrences in the dataset (pick the one with the highest)
- Map out the relationship flow from part to other parts that they go with and labour involved
- Then work on 

In [261]:
GOLF_Parts_Lines_df["PL.fldPartNumber"].value_counts().head(30)

PL.fldPartNumber
ENV55                6594
ENV5                 6297
06L 103 801          3890
N 908 132 02         3668
06L 115 562 B        3400
06D 115 562          1996
ZVW 352 540 S DSP    1890
G E55 540 1B DSP     1874
G E52 502 1B DSP     1794
84032                1184
TL                   1095
5QM 129 620          1087
5Q0 819 669           985
071 115 562 C         949
K5902                 945
06K 115 562           849
N 013 968 1           826
ZVW 817 001           779
06L 103 801 D         689
1J0 919 133 B         684
G E52 167 1B DSP      659
G E60 020 1Q DSP      640
--- VAN ---           606
61                    582
G 013 A8J 1G          553
5G1 061 550 041       547
B 000 750 1L DSP      538
06L 115 562           525
5G1 713 456 3DP       523
1K1 819 653 B         492
Name: count, dtype: int64

In [262]:
GOLF_Parts_Lines_df.columns

Index(['RO.fldID', 'REQ.fldId', 'PL.fldID,', 'PL.fldPartNumber', 'PL.fldCost',
       'PL.fldPartRef', 'PL.fldShipped', 'PL.fldPrice', 'PL.fldPartDesc'],
      dtype='object')

In [263]:

def search_part_by_partNumber(partNumber):

    search_response_part = GOLF_Parts_Lines_df[GOLF_Parts_Lines_df["PL.fldPartNumber"]== partNumber] # Get all records where this part was used
 
    search_response_part.reset_index(inplace=True, drop =True) # Reset index

    uniq_RO_IDs = search_response_part["RO.fldID"].unique()

    # Look up all parts that were used with our subject part
    all_parts_used_with_subject_part_df = GOLF_Parts_Lines_df[GOLF_Parts_Lines_df["RO.fldID"].isin(uniq_RO_IDs)]

    all_parts_used_with_subject_part_df = all_parts_used_with_subject_part_df[all_parts_used_with_subject_part_df["PL.fldPartNumber"] != partNumber]

    return all_parts_used_with_subject_part_df
 

In [264]:
def search_parts_used_with_byPartNumber(partNumber, percBaseline):
    parts_list_wit_prec = get_percentange_presence_parts(search_part_by_partNumber(partNumber))
    search_results = parts_list_wit_prec[parts_list_wit_prec["% Occurrence"]>percBaseline]

    return search_results


In [265]:
GOLF_Parts_Lines_df.columns

Index(['RO.fldID', 'REQ.fldId', 'PL.fldID,', 'PL.fldPartNumber', 'PL.fldCost',
       'PL.fldPartRef', 'PL.fldShipped', 'PL.fldPrice', 'PL.fldPartDesc'],
      dtype='object')

In [266]:
GOLF_Parts_Lines_df.columns

Index(['RO.fldID', 'REQ.fldId', 'PL.fldID,', 'PL.fldPartNumber', 'PL.fldCost',
       'PL.fldPartRef', 'PL.fldShipped', 'PL.fldPrice', 'PL.fldPartDesc'],
      dtype='object')

In [267]:
GOLF_Parts_Lines_df["PL.fldPartDesc"]

0        PIR PZERONERO
1            TIRE LEVY
2                SCREW
3           WASHERFLUI
4           NITRO PACK
             ...      
98799       ENVIRO FEE
98800       FILTERELEM
98801       ENVIRO FEE
98802       drain plug
98803       Dust filt.
Name: PL.fldPartDesc, Length: 98804, dtype: object

In [268]:
def get_top_15_busy_parts():
    return    GOLF_Parts_Lines_df["PL.fldPartNumber"].value_counts().head(15)
 

In [269]:
search_parts_used_with_byPartNumber("N 908 132 02", 10)

,PL.fldPartNumber,fldPartDesc,JobCount,% Occurrence
0,ENV55,ENVIRO FEE,2046,56.116292
1,ENV5,ENVIRO FEE,2022,55.458036
2,06D 115 562,FILTERELEM,1738,47.668678
3,ZVW 352 540 S DSP,ENGINE OIL,951,26.083379
4,071 115 562 C,FILTERELEM,796,21.832145
5,61,BASIC SERVICE,453,12.424575
6,G E60 020 1Q DSP,ENG.OIL,441,12.095447
7,84032,BG BRAKE FLUID,423,11.601755
8,07K 129 620,AIRFILTER,400,10.970927
9,G E52 502 1B DSP,EDGE,381,10.449808


In [271]:
GOLF_Parts_Lines_df[GOLF_Parts_Lines_df["PL.fldPartDesc"].str.contains("Trans", case=False, na=False)]["PL.fldPartDesc"].value_counts()


PL.fldPartDesc
TRANSM.OIL                                  109
Transmission Service Kit                     18
TRANSMITTE                                   14
TRANSMISSN                                    7
USED TRANSMISSION                             3
Used Transmission with 56k kms                1
quote me 3L of manual transmssion fluid.      1
A/M Transmission Mount                        1
quote me 3L manual transmison fluid           1
Name: count, dtype: int64

In [ ]:
GOLF_Parts_Lines_df[
    (GOLF_Parts_Lines_df["PL.fldPartDesc"].str.contains("Trans", case=False, na=False)) &
    (~GOLF_Parts_Lines_df["PL.fldPartDesc"].str.contains(r"(?:km|kit|oil|fluid)", case=False, na=False))
]

RO.fldID            25
REQ.fldId           25
PL.fldID,           25
PL.fldPartNumber    25
PL.fldCost          25
PL.fldPartRef       25
PL.fldShipped       25
PL.fldPrice         25
PL.fldPartDesc      25
dtype: int64

Anlysis of partline with Keyword Water Pump



In [102]:
def find_part_from_RO_list(RO_list):
    return GOLF_Parts_Lines_df[GOLF_Parts_Lines_df["RO.fldID"].isin(RO_list)]

In [103]:
uniq_Ro_w_word_WaterPump = GOLF_Parts_Lines_df[GOLF_Parts_Lines_df["PL.fldPartDesc"].str.contains(r"(?:water Pump)", case=False, na=False)]["RO.fldID"]

In [105]:
jobs_with_water_pump_df = find_part_from_RO_list(uniq_Ro_w_word_WaterPump)

In [ ]:
get_percentange_presence_parts(jobs_with_water_pump_df)

,PL.fldPartNumber,fldPartDesc,JobCount,% Occurrence
0,G 013 A8J 1G,COOL.ADD.,86,42.364532
1,ENV55,ENVIRO FEE,82,40.394089
2,ENV5,ENVIRO FEE,82,40.394089
3,N 905 969 06,BOLT,66,32.512315
4,N 910 296 02,BOLT,60,29.556650
5,N 102 096 05,BOLT,59,29.064039
6,N 103 280 02,SCREW,59,29.064039
7,N 907 124 03,BOLT,58,28.571429
8,N 908 132 02,SCREW,53,26.108374
9,N 105 524 02,BOLT,48,23.645320
